In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
test = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

train.shape, test.shape

### O modelo de ML será realizado com base nos dados "Airline Passenger Satisfaction", os quais, com uma série de variáveis, apresentam a satisfação dos passageiros em relação ao vôo. Inicialmente, observou-se que a base de dados possue 129880 linhas e 25 colunas e, após isso, foram observadas as 10 primeiras linhas dos dados, para uma observação inicial de como estes se comportam.

In [ ]:
df = train.append(test)
df.shape

In [ ]:
df.head(10)

### Foi observada uma coluna "Unnamed: 0" que representa um índice e não possui utilidade para a análise. Sendo assim, esta foi removida.

In [ ]:
df.info()

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df_2 = df.copy()

### Foram criadas dummies para as variáveis "Gender", "Customer Type", "Type of Travel", "Class" e a variável target "satisfaction". Sendo assim, para esta última, 0 representa que o passageiro estava neutro ou insatisfeito e 1 que o passageiro estava satisfeito com o vôo.

In [ ]:
df_2.select_dtypes('object').head()

In [ ]:
df_2 = pd.get_dummies(df_2,drop_first=True)
df_2.head(5)

### A variável "Arrival Delay in Minutes" apresentou 393 valores nulos e, através da análise dos dados, constatou-se que não há nenhum padrão aparente que indique o porquê destes valores nulos. Além disso, não foram encontradas observações duplicadas na base.

In [ ]:
df_2.isna().sum()

In [ ]:
df_2[df_2.isnull().any(axis=1)]

In [ ]:
df_2.id.duplicated().sum()

### Através da análise das maiores correlações encontradas na base, foi observado que as variáveis "Arrival Delay in Minutes" e "Departure Delay in Minutes" possuem uma correlação de aproximadamente 97%. Portanto, para o modelo de ML, apenas a variável "Departure Delay in Minutes" será considerada, já que esta não apresenta valores nulos. 

In [ ]:
import matplotlib.pyplot as plt

plt.scatter('Departure Delay in Minutes', 'satisfaction_satisfied', data=df_2)

In [ ]:
plt.scatter('Arrival Delay in Minutes', 'satisfaction_satisfied', data=df_2)

In [ ]:
c = df_2.corr().abs()

s = c.unstack()
so = s.sort_values(kind="quicksort")
print(so[(so>0.5)&(so<1)])

### A variável target está bem distribuida entre 0 e 1. Porém, há de se observar que, para a variável "Departure Delay in Minutes", existem alguns vôos que apresentaram um atraso bastante elevado quando comparado à maioria dos outros vôos. 

In [ ]:
df_2.satisfaction_satisfied.value_counts(normalize=True)

In [ ]:
df_2.describe()

In [ ]:
df_2 = df_2.rename(columns={'Departure Delay in Minutes':'departure_delay'})

In [ ]:
plt.hist(df_2.departure_delay,alpha=0.8, color='tab:blue')

plt.show()

In [ ]:
df_2[df_2['departure_delay']>500].sort_values('departure_delay')

### Nos próximos passos, o modelo será executado na base de treino e a análise dos resultados será realizada na base de teste.

In [ ]:
test = df_2[df_2.id.isin(test.id)]
train = df_2[~df_2.id.isin(test.id)]

test.shape, train.shape

In [ ]:
removed_cols = ['id', 'Arrival Delay in Minutes', 'satisfaction_satisfied']

feats = [c for c in train.columns if c not in removed_cols]
feats

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(random_state=42, n_estimators=200)
rf.fit(train[feats], train['satisfaction_satisfied'])

In [ ]:
preds_test = rf.predict(test[feats])

preds_test

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(test['satisfaction_satisfied'], preds_test)

In [ ]:
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

### O modelo Random Forest obteve um ótimo desempenho em prever a satisfação dos clientes em relação ao vôo, apresentando uma acurácia de aproximadamente 96%. As variáveis que tiveram um maior impacto na satisfação dos passageiros foram aquelas relacionadas aos serviços fornecidos no avião, dentre estes, "Online boarding" e "Inflight wifi service". Além disso, o tipo de viagem é relevante na satisfação do cliente, assim como a classe em que este se encontra. 